# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f708419fcd0>
├── 'a' --> tensor([[-0.9171,  1.7155,  1.3799],
│                   [-0.2327, -0.3378, -0.5759]])
└── 'x' --> <FastTreeValue 0x7f708419fca0>
    └── 'c' --> tensor([[-0.1696, -0.5605,  1.7564, -1.4193],
                        [ 1.4972,  0.3880, -0.9034,  0.0131],
                        [ 0.4925,  1.1353, -1.1177, -0.8582]])

In [4]:
t.a

tensor([[-0.9171,  1.7155,  1.3799],
        [-0.2327, -0.3378, -0.5759]])

In [5]:
%timeit t.a

59.1 ns ± 0.0899 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f708419fcd0>
├── 'a' --> tensor([[0.1999, 0.3717, 0.6863],
│                   [0.7031, 0.6198, 0.6170]])
└── 'x' --> <FastTreeValue 0x7f708419fca0>
    └── 'c' --> tensor([[-0.1696, -0.5605,  1.7564, -1.4193],
                        [ 1.4972,  0.3880, -0.9034,  0.0131],
                        [ 0.4925,  1.1353, -1.1177, -0.8582]])

In [7]:
%timeit t.a = new_value

59.8 ns ± 0.0467 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9171,  1.7155,  1.3799],
               [-0.2327, -0.3378, -0.5759]]),
    x: Batch(
           c: tensor([[-0.1696, -0.5605,  1.7564, -1.4193],
                      [ 1.4972,  0.3880, -0.9034,  0.0131],
                      [ 0.4925,  1.1353, -1.1177, -0.8582]]),
       ),
)

In [10]:
b.a

tensor([[-0.9171,  1.7155,  1.3799],
        [-0.2327, -0.3378, -0.5759]])

In [11]:
%timeit b.a

58.5 ns ± 0.0285 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7164, -0.8162, -0.5063],
               [-0.0230, -0.1833,  0.6759]]),
    x: Batch(
           c: tensor([[-0.1696, -0.5605,  1.7564, -1.4193],
                      [ 1.4972,  0.3880, -0.9034,  0.0131],
                      [ 0.4925,  1.1353, -1.1177, -0.8582]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.123 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

855 ns ± 0.109 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 58.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 764 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6fd60afd60>
├── 'a' --> tensor([[[-0.9171,  1.7155,  1.3799],
│                    [-0.2327, -0.3378, -0.5759]],
│           
│                   [[-0.9171,  1.7155,  1.3799],
│                    [-0.2327, -0.3378, -0.5759]],
│           
│                   [[-0.9171,  1.7155,  1.3799],
│                    [-0.2327, -0.3378, -0.5759]],
│           
│                   [[-0.9171,  1.7155,  1.3799],
│                    [-0.2327, -0.3378, -0.5759]],
│           
│                   [[-0.9171,  1.7155,  1.3799],
│                    [-0.2327, -0.3378, -0.5759]],
│           
│                   [[-0.9171,  1.7155,  1.3799],
│                    [-0.2327, -0.3378, -0.5759]],
│           
│                   [[-0.9171,  1.7155,  1.3799],
│                    [-0.2327, -0.3378, -0.5759]],
│           
│                   [[-0.9171,  1.7155,  1.3799],
│                    [-0.2327, -0.3378, -0.5759]]])
└── 'x' --> <FastTreeValue 0x7f6fd60afd00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 26.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6fd60afa00>
├── 'a' --> tensor([[-0.9171,  1.7155,  1.3799],
│                   [-0.2327, -0.3378, -0.5759],
│                   [-0.9171,  1.7155,  1.3799],
│                   [-0.2327, -0.3378, -0.5759],
│                   [-0.9171,  1.7155,  1.3799],
│                   [-0.2327, -0.3378, -0.5759],
│                   [-0.9171,  1.7155,  1.3799],
│                   [-0.2327, -0.3378, -0.5759],
│                   [-0.9171,  1.7155,  1.3799],
│                   [-0.2327, -0.3378, -0.5759],
│                   [-0.9171,  1.7155,  1.3799],
│                   [-0.2327, -0.3378, -0.5759],
│                   [-0.9171,  1.7155,  1.3799],
│                   [-0.2327, -0.3378, -0.5759],
│                   [-0.9171,  1.7155,  1.3799],
│                   [-0.2327, -0.3378, -0.5759]])
└── 'x' --> <FastTreeValue 0x7f708419fdc0>
    └── 'c' --> tensor([[-0.1696, -0.5605,  1.7564, -1.4193],
                        [ 1.4972,  0.3880, -0.9034,  0.0131],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 45.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.1 µs ± 77 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9171,  1.7155,  1.3799],
                [-0.2327, -0.3378, -0.5759]],
       
               [[-0.9171,  1.7155,  1.3799],
                [-0.2327, -0.3378, -0.5759]],
       
               [[-0.9171,  1.7155,  1.3799],
                [-0.2327, -0.3378, -0.5759]],
       
               [[-0.9171,  1.7155,  1.3799],
                [-0.2327, -0.3378, -0.5759]],
       
               [[-0.9171,  1.7155,  1.3799],
                [-0.2327, -0.3378, -0.5759]],
       
               [[-0.9171,  1.7155,  1.3799],
                [-0.2327, -0.3378, -0.5759]],
       
               [[-0.9171,  1.7155,  1.3799],
                [-0.2327, -0.3378, -0.5759]],
       
               [[-0.9171,  1.7155,  1.3799],
                [-0.2327, -0.3378, -0.5759]]]),
    x: Batch(
           c: tensor([[[-0.1696, -0.5605,  1.7564, -1.4193],
                       [ 1.4972,  0.3880, -0.9034,  0.0131],
                       [ 0.4925,  1.1353, -1.1177, -0.8582]],
         

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9171,  1.7155,  1.3799],
               [-0.2327, -0.3378, -0.5759],
               [-0.9171,  1.7155,  1.3799],
               [-0.2327, -0.3378, -0.5759],
               [-0.9171,  1.7155,  1.3799],
               [-0.2327, -0.3378, -0.5759],
               [-0.9171,  1.7155,  1.3799],
               [-0.2327, -0.3378, -0.5759],
               [-0.9171,  1.7155,  1.3799],
               [-0.2327, -0.3378, -0.5759],
               [-0.9171,  1.7155,  1.3799],
               [-0.2327, -0.3378, -0.5759],
               [-0.9171,  1.7155,  1.3799],
               [-0.2327, -0.3378, -0.5759],
               [-0.9171,  1.7155,  1.3799],
               [-0.2327, -0.3378, -0.5759]]),
    x: Batch(
           c: tensor([[-0.1696, -0.5605,  1.7564, -1.4193],
                      [ 1.4972,  0.3880, -0.9034,  0.0131],
                      [ 0.4925,  1.1353, -1.1177, -0.8582],
                      [-0.1696, -0.5605,  1.7564, -1.4193],
                      [ 1.4972,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 233 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

314 µs ± 2.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
